In [1]:
!pip install google-api-python-client

In [39]:
!pip install pymongo[srv]

     |████████████████████████████████| 194kB 2.8MB/s 


In [0]:
from googleapiclient.discovery import build
#requests da lib no lambda - para rodar localmente -> import requests
import requests
# from botocore.vendored import requests
from pymongo import MongoClient
from datetime import datetime

In [0]:
my_api_key = "##############################"
my_cse_id = "##############################"

client = MongoClient("##############################")
db = client.boavenda

collection_busca_site_cliente = db['BuscaSiteCliente']
collection_resultado_busca_google = db['ResultadoBuscaGoogle']

def google_search(search_term, api_key, cse_id, start):
    r = requests.get('https://customsearch.googleapis.com/customsearch/v1?q=' + search_term + '&cx=' + cse_id + '&key=' + api_key + '&start=' + str(start) + '&gl=br&lr=lang_pt')
    if 'items' in r.json():
      return r.json()['items']
    else:
      return []

#buscar no mongo e loopar
cursor = collection_busca_site_cliente.find()
buscas = list(cursor)

for b in buscas:
  link_busca = b['link']
  busca = b['busca']

  results = []
  for i in range(0,5):
    results.extend(google_search(busca, my_api_key, my_cse_id, i*10+1))

  for i, result in enumerate(results):
    if link_busca in result['link']:
      dados = {}

      dados['posicaoGoogle'] = i+1

      try:
        dados['buscaId'] = b['buscaId']
      except:
        dados['buscaId'] = b['_id']

      try:
        dados['preco'] = float(result['pagemap']['offer'][0]['price'])
      except:
        dados['preco'] = None

      try:
        dados['imagem'] = result['pagemap']['cse_image'][0]['src']
      except:
        dados['imagem'] = None

      try:
        dados['review'] = result['pagemap']['review']
      except:
        dados['review'] = None

      dados['dtIns'] = datetime.utcnow()
      collection_resultado_busca_google.insert_one(dados)


In [0]:
busca = 'notebook samsung x55'
r = requests.get('https://customsearch.googleapis.com/customsearch/v1?q=' + busca + '&cx=' + my_cse_id + '&key=' + my_api_key + '&start=' + str(1) + '&gl=br&lr=lang_pt')
resultado = r.json()['items']

In [0]:
results = []
for i in range(0,5):
  results.extend(google_search(busca, my_api_key, my_cse_id, i*10+1))

In [30]:
len(results)

31

In [29]:
results

[{'cacheId': 'CCD2MZfDutcJ',
  'displayLink': 'www.magazineluiza.com.br',
  'formattedUrl': 'https://www.magazineluiza.com.br/notebook-samsung...x55.../ssex/',
  'htmlFormattedUrl': 'https://www.magazineluiza.com.br/<b>notebook</b>-<b>samsung</b>...<b>x55</b>.../ssex/',
  'htmlSnippet': '<b>Notebook Samsung</b> Expert <b>X55</b> Intel Core i7 16GB - 1TB 128GB SSD 15,6” <br>\nNVIDIA MX110 Windows 10 Home com as melhores condições você encontra no<br>\n&nbsp;...',
  'htmlTitle': '<b>Notebook Samsung</b> Expert <b>X55</b> Intel Core i7 16GB - 1TB 128GB ...',
  'kind': 'customsearch#result',
  'link': 'https://www.magazineluiza.com.br/notebook-samsung-expert-x55-intel-core-i7-16gb-1tb-128gb-ssd-156-nvidia-mx110-windows-10-home/p/224714900/in/ssex/',
  'pagemap': {'aggregaterating': [{'ratingvalue': '3,6', 'reviewcount': '7'},
    {'ratingvalue': '3,6', 'reviewcount': '7'}],
   'brand': [{'name': 'Samsung'}],
   'cse_image': [{'src': 'https://a-static.mlcdn.com.br/470x352/notebook-samsung-e

In [22]:
resultado[0]['pagemap']['offer']

[{'availability': 'InStock', 'price': '4464.05', 'pricecurrency': 'BRL'}]

In [21]:
resultado[0]

{'cacheId': 'CCD2MZfDutcJ',
 'displayLink': 'www.magazineluiza.com.br',
 'formattedUrl': 'https://www.magazineluiza.com.br/notebook-samsung...x55.../ssex/',
 'htmlFormattedUrl': 'https://www.magazineluiza.com.br/<b>notebook</b>-<b>samsung</b>...<b>x55</b>.../ssex/',
 'htmlSnippet': '<b>Notebook Samsung</b> Expert <b>X55</b> Intel Core i7 16GB - 1TB 128GB SSD 15,6” <br>\nNVIDIA MX110 Windows 10 Home com as melhores condições você encontra no<br>\n&nbsp;...',
 'htmlTitle': '<b>Notebook Samsung</b> Expert <b>X55</b> Intel Core i7 16GB - 1TB 128GB ...',
 'kind': 'customsearch#result',
 'link': 'https://www.magazineluiza.com.br/notebook-samsung-expert-x55-intel-core-i7-16gb-1tb-128gb-ssd-156-nvidia-mx110-windows-10-home/p/224714900/in/ssex/',
 'pagemap': {'aggregaterating': [{'ratingvalue': '3,6', 'reviewcount': '7'},
   {'ratingvalue': '3,6', 'reviewcount': '7'}],
  'brand': [{'name': 'Samsung'}],
  'cse_image': [{'src': 'https://a-static.mlcdn.com.br/470x352/notebook-samsung-expert-x55-in

In [0]:
client = MongoClient("##############################")
db = client.boavenda

collection_busca_site_cliente = db['BuscaSiteCliente']
collection_resultado_busca_google = db['ResultadoBuscaGoogle']

In [0]:
cursor = collection_busca_site_cliente.find()

In [7]:
list(cursor)

[{'_id': ObjectId('5eb082ee77fc2ff62d1fe68e'),
  'busca': 'notebook samsung x55',
  'buscaId': 1,
  'dtIns': datetime.datetime(2020, 4, 30, 20, 49, 29),
  'link': 'https://www.samsung.com/br/pc/notebook-expert-x55-np350xaa-xf4br/',
  'mecanismos': ['google', 'mercadolivre']},
 {'_id': ObjectId('5eb082ee77fc2ff62d1fe68f'),
  'busca': 'notebook samsung x55',
  'buscaId': 2,
  'dtIns': datetime.datetime(2020, 4, 30, 20, 49, 29),
  'link': 'https://www.samsung.com/br/pc/notebook-expert-x55-np350xbe-xh4br/',
  'mecanismos': ['google', 'mercadolivre']}]